Created on February 14th 2021 by Patrick Rotzetter

https://www.linkedin.com/in/rotzetter/

# Small experiment of document mining with various techniques Part 7

Let us try T5 summarization capabilities ( example for summarization function from denis Rothman book Transformers for Natural Language Processing

## Load the files

In [1]:
# Import require libraries
import numpy as np
import PyPDF2
import pandas as pd
import re
from pptx import Presentation
import pdftotext
import texthero as hero

In [2]:
# function to read PDF files using pdftotext
def readPdfFile(filename):
    text=""
    with open(filename, "rb") as f:
        pdf = pdftotext.PDF(f)
        for page in pdf:
            text=text+page
    return text

In [3]:
# function to read PPT files
def readPPTFile(filename):
    text=""  
    prs = Presentation(filename)
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                text=text+shape.text
    text=remove_special_characters(text)
    return text

In [4]:
#path of first input test file
path='./sampledocs/'

In [5]:
# let us scan the full directory, read PDF and PPT documents, clean them and process them with spacy

docName=[]
docType=[]
docText=[]
docNLP=[]
import glob
list_of_files = glob.glob(path+'*.pdf')           # create the list of file
fileNames=[]
for file_name in list_of_files:
    fileText=readPdfFile(file_name)
    docName.append(file_name)
    docType.append('pdf')
    docText.append(fileText)
list_of_files = glob.glob(path+'*.pptx')           # create the list of file
for file_name in list_of_files:
    fileText=readPPTFile(file_name)
    docName.append(file_name)
    docType.append('ppt')
    docText.append(fileText)
fullDocs = pd.DataFrame({'Name':docName,'Type':docType,'Text':docText})
fullDocs['cleanText']=hero.clean(fullDocs['Text'])


/opt/anaconda3/envs/spacy30/lib/python3.8/site-packages/texthero/preprocessing.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  return input.str.replace(pattern, symbols)
/opt/anaconda3/envs/spacy30/lib/python3.8/site-packages/texthero/preprocessing.py:131: FutureWarning: The default value of regex will change from True to False in a future version.
  return input.str.replace(rf"([{string.punctuation}])+", symbol)


In [6]:
 print ("Average length of text:" + str((np.mean(fullDocs['Text'].str.len()))))
 print ("Min length of text:" + str((np.min(fullDocs['Text'].str.len()))))
 print ("Max length of text:" + str((np.max(fullDocs['Text'].str.len()))))

Average length of text:154163.0
Min length of text:17987
Max length of text:464271


In [7]:
fullDocs['text_word_count'] = fullDocs['Text'].apply(lambda x: len(x.strip().split()))  # word count
fullDocs['text_unique_words']=fullDocs['Text'].apply(lambda x:len(set(str(x).split())))  # number of unique words
fullDocs.head()

,Name,Type,Text,cleanText,text_word_count,text_unique_words
0,./sampledocs/ai-360-research.pdf,pdf,AI 360: insights from the\nnext frontier of bu...,ai insights next frontier business corner offi...,5289,1752
1,./sampledocs/Module-1-Lecture-Slides.pdf,pdf,"Application of AI, Insurtech and Real Estate\n...",application ai insurtech real estate technolog...,3732,1509
2,./sampledocs/Technology-and-innovation-in-the-...,pdf,Technology and\ninnovation in the\ninsurance s...,technology innovation insurance sector technol...,16763,4237
3,./sampledocs/AI-bank-of-the-future-Can-banks-m...,pdf,Global Banking & Securities\n\n\n...,global banking securities ai bank future banks...,5804,2165
4,./sampledocs/sigma-5-2020-en.pdf,pdf,No 5 /2020\n\n\n\n\n...,machine intelligence executive summary machine...,14512,4342


In [8]:
fullDocs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               13 non-null     object
 1   Type               13 non-null     object
 2   Text               13 non-null     object
 3   cleanText          13 non-null     object
 4   text_word_count    13 non-null     int64 
 5   text_unique_words  13 non-null     int64 
dtypes: int64(2), object(4)
memory usage: 752.0+ bytes


In [9]:
fullDocs.describe()

,text_word_count,text_unique_words
count,13.000000,13.000000
mean,15731.769231,3924.384615
std,13745.310559,2412.564905
min,2502.000000,1006.000000
25%,5289.000000,1752.000000
50%,14512.000000,3685.000000
75%,18851.000000,5320.000000
max,49779.000000,8462.000000


## Test T5 summarization

In [10]:
# load libraries
import torch
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [11]:
model = T5ForConditionalGeneration.from_pretrained('t5-large')
tokenizer = T5Tokenizer.from_pretrained('t5-large')
device = torch.device('cpu')

Some weights of the model checkpoint at t5-large were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
print(model.config)

T5Config {
  "_name_or_path": "t5-large",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 4096,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to German: "
    },
    "translation_en_to_fr": {
      "early_stopping": t

In [14]:
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (

## Define a summarization function

In [12]:
def summarize(text,ml):
  preprocess_text = text.strip().replace("\n","")
  t5_prepared_Text = "summarize: "+preprocess_text
  print ("Preprocessed and prepared text: \n", t5_prepared_Text)

  tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)

  # summmarize 
  summary_ids = model.generate(tokenized_text,
                                      num_beams=4,
                                      no_repeat_ngram_size=2,
                                      min_length=30,
                                      max_length=ml,
                                      early_stopping=True)

  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return output

## Let us test the function on a short text

In [13]:
text="""
The United States Declaration of Independence was the first Etext
released by Project Gutenberg, early in 1971.  The title was stored
in an emailed instruction set which required a tape or diskpack be
hand mounted for retrieval.  The diskpack was the size of a large
cake in a cake carrier, cost $1500, and contained 5 megabytes, of
which this file took 1-2%.  Two tape backups were kept plus one on
paper tape.  The 10,000 files we hope to have online by the end of
2001 should take about 1-2% of a comparably priced drive in 2001.
"""
print("Number of characters:",len(text))
summary=summarize(text,50)
print ("\n\nSummarized text: \n",summary)


Number of characters: 534
Preprocessed and prepared text: 
 summarize: The United States Declaration of Independence was the first Etextreleased by Project Gutenberg, early in 1971.  The title was storedin an emailed instruction set which required a tape or diskpack behand mounted for retrieval.  The diskpack was the size of a largecake in a cake carrier, cost $1500, and contained 5 megabytes, ofwhich this file took 1-2%.  Two tape backups were kept plus one onpaper tape.  The 10,000 files we hope to have online by the end of2001 should take about 1-2% of a comparably priced drive in 2001.


Summarized text: 
 the united states declaration of independence was the first etext published by project gutenberg, early in 1971. the 10,000 files we hope to have online by the end of2001 should take about 1-2% of a comparably priced drive in


## Let us now apply the summarization function to one of the loaded texts

In [18]:
fullDocs.loc[3,'cleanText']

'global banking securities ai bank future banks meet ai challenge artificial intelligence technologies increasingly integral world live banks need deploy technologies scale remain relevant success requires holistic transformation spanning multiple layers organization suparna biswas brant carson violet chung shwaitang singh renny thomas (c) getty images september alphago machine defeated time obstacles prevent banks deploying world champion lee sedol game ai capabilities scale go complex board game requiring intuition imagination strategic thinking--abilities banks transform become ai first long considered distinctly human since artificial intelligence ai technologies advanced even 1 transformative must banks become ai first impact increasingly evident across several decades banks continually industries ai powered machines tailoring adapted latest technology innovations recommendations digital content individual redefine customers interact banks tastes preferences designing clothing int

In [15]:
summary=summarize(fullDocs.loc[3,'cleanText'],200)


Token indices sequence length is longer than the specified maximum sequence length for this model (5461 > 512). Running this sequence through the model will result in indexing errors


Preprocessed and prepared text: 
 summarize: global banking securities ai bank future banks meet ai challenge artificial intelligence technologies increasingly integral world live banks need deploy technologies scale remain relevant success requires holistic transformation spanning multiple layers organization suparna biswas brant carson violet chung shwaitang singh renny thomas (c) getty images september alphago machine defeated time obstacles prevent banks deploying world champion lee sedol game ai capabilities scale go complex board game requiring intuition imagination strategic thinking--abilities banks transform become ai first long considered distinctly human since artificial intelligence ai technologies advanced even 1 transformative must banks become ai first impact increasingly evident across several decades banks continually industries ai powered machines tailoring adapted latest technology innovations recommendations digital content individual redefine customers interact ban

In [16]:
print(summary)

mckinsey's global ai playbook reveals banks' need to become ii first banks. banks must deploy technologies across four layers to achieve requisite agility and scale if they are to compete successfully in the omnichannel banking market dominated by fintechs and tech giants. the bank must be able to deliver personalized experiences to customers across multiple channels and channels without relying on centralized technology systems.


## The result is quite amazing!